# Analyse de données financières : Détection d'anomalies dans un porteuille d'actifs

In [ ]:
import quandl
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

from sklearn.ensemble import IsolationForest

# instruction pour plotter dans notebook
%matplotlib inline

# connection à quandl via l'api key
# quandl.ApiConfig.api_key = " 3CipYF1Y3fzjvDgg7E2n"

In [206]:
# Lecture des données
# df = pd.read_csv("all_stocks_5yr.csv", index_col= [0,2],usecols=[0,4,6],parse_dates=[0])
df = pd.read_csv("all_stocks_5yr.csv",usecols=[0,4,6], parse_dates=[0])
df.head()
df.Date = pd.to_datetime(df.Date)
df = df.loc[(df.Date<datetime.datetime(2017, 8, 11,0,0,0)),:]
df = df.loc[df.Date>datetime.datetime(2015, 8, 11,0,0,0),:]
#df.loc[df.Name=='MMM',:]

In [207]:
df = df.set_index(['Name','Date'])

In [ ]:
# On récupère les donnée sur deux ans, les données principales sont des données sur cinq ans


In [209]:
df_unstack = df.unstack(level=-1)
df_unstack

Close                                                         \
Date 2015-08-12 2015-08-13 2015-08-14 2015-08-17 2015-08-18 2015-08-19   
Name                                                                     
A         38.89      38.56      38.65      38.82      39.02      38.40   
AAL       42.52      42.67      42.88      43.95      43.63      43.53   
AAP      172.00     187.79     187.03     184.69     185.77     187.84   
AAPL     115.24     115.15     115.96     117.16     116.50     115.01   
ABBV      68.75      68.54      68.65      69.37      68.86      68.59   
ABC      104.38     103.88     103.94     104.62     104.95     106.14   
ABT       50.09      49.84      50.23      50.38      50.44      49.84   
ACN      103.77     103.39     103.45     103.48     103.86     102.87   
ADBE      83.82      84.31      86.12      86.77      86.45      85.48   
ADI       59.74      59.00      58.84      59.50      57.90      58.38   
ADM       46.89      47.51      47.20      47.51      47.68      46.72   
ADP       81.55      82.45      82.99      84.70      84.69      84.47   
ADS      269.53     268.92     273.78     271.32     273.49     268.57   
ADSK      54.82      54.61      54.86      55.40      54.59      54.32   
AEE       42.65      42.62      42.74      43.40      43.20      43.55   
AEP       57.57      57.55      58.20      58.35      58.27      58.80   
AES       13.06      12.66      12.81      12.87      12.82      12.54   
AET      117.30     118.25     120.66     123.04     124.21     122.18   
AFL       64.11      64.17      64.76      64.53      64.58      63.05   
AGN      315.81     315.65     314.50     321.25     320.30     318.86   
AIG       63.26      63.24      64.25      64.30      63.71      62.92   
AIV       39.98      39.61      39.84      40.15      40.34      39.98   
AIZ       78.85      79.56      79.60      79.48      79.38      78.94   
AJG       47.17      47.03      47.30      47.71      47.67      47.12   
AKAM      73.92      73.84      73.37      73.57      73.85      73.06   
ALB       51.12      50.42      51.11      51.77      51.11      49.88   
ALGN      60.83      60.20      60.57      60.88      60.52      59.73   
ALK       78.54      79.29      79.60      81.51      81.51      80.99   
ALL       63.61      63.49      63.86      63.85      63.57      63.66   
ALLE      63.07      63.29      63.60      64.68      64.57      64.23   
...         ...        ...        ...        ...        ...        ...   
VTR       67.97      67.85      68.39      68.52      59.80      60.05   
VZ        47.85      47.66      47.49      47.52      47.46      47.44   
WAT      129.62     128.62     128.82     129.24     128.92     128.76   
WBA       93.36      92.54      92.10      92.70      91.94      91.41   
WDC       83.16      81.57      82.50      83.60      83.31      81.71   
WEC       50.96      50.96      51.34      51.29      51.27      51.53   
WFC       56.72      56.89      57.33      57.35      57.54      57.06   
WFM       34.27      33.72      33.31      33.69      33.36      32.79   
WHR      172.79     174.57     177.22     178.49     182.95     184.58   
WLTW     121.40     121.43     122.52     123.07     122.94     121.67   
WM        51.85      51.90      52.25      52.64      52.70      52.60   
WMB       51.11      51.32      52.23      54.90      53.86      53.01   
WMT       72.58      72.11      72.38      71.91      69.48      68.57   
WRK       63.31      63.13      63.23      63.34      63.16      62.07   
WU        20.42      20.38      20.35      20.42      20.38      19.95   
WY        30.95      30.99      31.19      31.08      31.15      30.91   
WYN       80.08      80.40      79.03      80.60      80.35      80.42   
WYNN      97.45      94.62      92.17      91.91      90.90      90.25   
XEC      118.80     115.35     116.34     115.28     117.29     112.39   
XEL       35.72      35.56      35.80      35.99      35.91      36.32   
XL          Na

## Rendements

 DataFrame.diff(periods=1, axis=0)[source]

    1st discrete difference of object
    Parameters:	

    periods : int, default 1

        Periods to shift for forming difference

    axis : {0 or ‘index’, 1 or ‘columns’}, default 0

        Take difference over rows (0) or columns (1).

    Returns:	

    diffed : DataFrame

In [ ]:
# 

rendments = all_data/all_data.diff(1) - 1

## Moyennes mobiles

In [ ]:
# Calculate the moving average
moving_avg = all_data.rolling(window=40).mean()

## Volatilités

In [ ]:
# volatilité

# Define the minumum of periods to consider 
min_periods = 75 

# Calculate the volatility
vol = daily_pct_change.rolling(min_periods).std() * np.sqrt(min_periods) 

# Plot the volatility
vol.plot(figsize=(10, 8))

# Show the plot
plt.show()

# Detection d'anomalies

In [ ]:
# df_2 est de dataframe qui contient les données dont on doit détecter les anomalies
# Applications de l'isolation forest sur les données de rendements ou de volatilités
clf = IsolationForest(n_estimator = 100, bootstrap = True)

# fit de l'estimateur
clf.fit(df_2)

# the anomaly score of the input samples. the lower the more abnormal.
scores_pred = clf.decision_fonction(df_2)

In [ ]:
# Détection d'anomalies sur les actifs

